# Decorators : closure, variable scope, nonlocal...

---------------------

source : "Fluent Python" by Luciano Ramalho

---------------------

## Decorators 101

Definition : a decorator is a callable that takes another function as argument (the decorated function). The decorator may perform some processing with the decorated function, and return it or replace it with another function or callable object.

Syntax and syntaxer

In [1]:
def decorate(func):
    print('It\'s decorate!')
    return func

In [2]:
@decorate
def target1():
    print('running target1()')
    
target1()

It's decorate!
running target1()


In [3]:
def target2():
    print('running target2()')
    
decorate(target2) # Not decorate(target2())
target2()

It's decorate!
running target2()


-------------

Here is a decorator that replaces a function with a different one.

In [4]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

In [5]:
target()

running inner()


In [6]:
target

<function __main__.deco.<locals>.inner()>

--------------------------------

## When Python executes decorators

![question pour un kahoot](assets/Kahoot_is_not_my_friend_anymore.png)

In [7]:
registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
    
if __name__ == '__main__':
    main()

running register(<function f1 at 0x7fc0fc39bd08>)
running register(<function f2 at 0x7fc0fc35f048>)
running main()
registry -> [<function f1 at 0x7fc0fc39bd08>, <function f2 at 0x7fc0fc35f048>]
running f1()
running f2()
running f3()


=> They run right after the decorated function is defined. That is usually at import time.

---------------------

In [ ]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_orger(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    return order.total() * .07 if len(distinct_items) >= 10 else 0

def best_promo(order):
    """Select best discount available"""
    return max(promo(order) for promo in promos)

--------------

Until now, our decorators don"t change the decorated function. To do so, we usually need to define an inner function and return it to replace the decorated function. Code that uses inner function almost always  depends on closures to operate correctly. And to understand closures, we need to look at how variables scopes work in Python.

----------------

## Variable Scope

In [8]:
def f1(a):
    print(a)
    print(b)
    
f1(3)

3


NameError: name 'b' is not defined

In [9]:
b = 6
f1(3)

3
6


In [10]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9
    
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

The generate bytecode is different.

In [11]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
    
f3(3)
b

3
6


9

---------------

## Closures

A closure is a function with an extended scope that encompasses nonglobal variables referenced in the body of the function but not define there.

##### An average function using oop

In [12]:
class Averager():
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [13]:
avg = Averager()
avg(10)

10.0

In [14]:
avg(11)

10.5

In [15]:
avg(12)

11.0

----------------------------

##### Same, but using a higher-order function

In [16]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager

In [17]:
avg = make_averager()
avg(10)

10.0

In [18]:
avg(11)

10.5

In [19]:
avg(12)

11.0

When invoked, make_averager returns an averager function object.

-----------------

Question is : where does this avg function find the series?

---------------

![Closures](assets/closures.svg)

Free variable : a variable that is not bound in the local scope.

In [20]:
avg.__code__.co_varnames

('new_value', 'total')

In [21]:
avg.__code__.co_freevars

('series',)

In [22]:
avg.__closure__

(<cell at 0x7fc0fe3cc888: list object at 0x7fc0fc2c2248>,)

Each item here corresponds to a name in co_freevars

In [23]:
avg.__closure__[0].cell_contents

[10, 11, 12]

------------------------

###### Conclusion about closure

A closure is a function that retains the bindings of the free variables that exist when the function is defined, so that they can be used later when the function is invoked and the defining scope is no longer available.

Note that the only situation in which a function may need to deal with external variables that are nonglobal is when it's nested in another function.

-------------------------

##### So, let's optimize our code !

In [24]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total/count
    
    return averager

In [25]:
avg = make_averager()

In [26]:
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [ ]:
avg(11)

In [ ]:
avg(12)

---------------------

## The nonlocal Declaration

nonlocal : lets you flag a variable as a free variable even when it's assigned a new value within the function.

Nonlocal variables are used in nested functions whose local scope is not defined. This means that the variable can be neither in the local nor the global scope.

In [27]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    
    return averager

In [28]:
avg = make_averager()
avg(10)

10.0

In [29]:
avg(11)

10.5

In [30]:
avg(12)

11.0

In [31]:
count

NameError: name 'count' is not defined

In [33]:
avg2 = make_averager()
avg2(5)

5.0

----------------

## Implementing a simple decorator

##### A decorator that output the running time of functions

In [34]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'{elapsed:.8f} {name}({arg_str}) -> {result}')
        return result
    return clocked

In [35]:
@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

In [36]:
print('*' * 25, 'Calling snooze(.125)')
snooze(.125)
print('*' * 25, 'Calling factorial(6)')
factorial(6)

************************* Calling snooze(.125)
0.12518814 snooze(0.125) -> None
************************* Calling factorial(6)
0.00000136 factorial(1) -> 1
0.00011032 factorial(2) -> 2
0.00097278 factorial(3) -> 6
0.00114291 factorial(4) -> 24
0.00123673 factorial(5) -> 120
0.00132642 factorial(6) -> 720


720

- clock gets factorial as its func argument
- It then creates and return the clocked function -> behind the scene, Python assigns it to factorial
- The factorial function is modified.

-----------------

In [37]:
factorial.__name__

'clocked'

Factorial now holds a reference to the clocked function. From now on, each time factorial(n) is called, clocked(n) gets executed.

This is the typical behavior of a decorator. It replaces the decorated function with a new function that accepts the same arguments.

##### But I kinda liked the name of my function without the decorator...

No worries, functools has our back.

In [38]:
import time
import functools

import time

def clock(func):
    @functools.wraps(func)
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'{elapsed:.8f} {name}({arg_str}) -> {result}')
        return result
    return clocked

In [39]:
@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

In [40]:
factorial.__name__

'factorial'

##### Another decorator in the standard library : functools.lru_cache

Memoization : an optimization technique that works by saving the results of previous invocations of an expensive function, avoiding repeat computations on previously used arguments.

LRU => Last Recently Used

In [41]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'{elapsed:.8f} {name}({arg_str}) -> {result}')
        return result
    return clocked

In [42]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [43]:
print(fibonacci(6))

0.00000059 fibonacci(0) -> 0
0.00000087 fibonacci(1) -> 1
0.00015599 fibonacci(2) -> 1
0.00000043 fibonacci(1) -> 1
0.00000071 fibonacci(0) -> 0
0.00000044 fibonacci(1) -> 1
0.00007945 fibonacci(2) -> 1
0.00029059 fibonacci(3) -> 2
0.00055374 fibonacci(4) -> 3
0.00000039 fibonacci(1) -> 1
0.00000058 fibonacci(0) -> 0
0.00000040 fibonacci(1) -> 1
0.00005398 fibonacci(2) -> 1
0.00010802 fibonacci(3) -> 2
0.00000043 fibonacci(0) -> 0
0.00000037 fibonacci(1) -> 1
0.00005369 fibonacci(2) -> 1
0.00000041 fibonacci(1) -> 1
0.00000049 fibonacci(0) -> 0
0.00000043 fibonacci(1) -> 1
0.00007637 fibonacci(2) -> 1
0.00013074 fibonacci(3) -> 2
0.00023629 fibonacci(4) -> 3
0.00039598 fibonacci(5) -> 5
0.00100365 fibonacci(6) -> 8
8


fibonacci(1) is called 8 times... => time wasted

In [44]:
import functools

@functools.lru_cache() # can take parameters
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [45]:
print(fibonacci(6))

0.00000053 fibonacci(0) -> 0
0.00000127 fibonacci(1) -> 1
0.00022253 fibonacci(2) -> 1
0.00000157 fibonacci(3) -> 2
0.00032352 fibonacci(4) -> 3
0.00000098 fibonacci(5) -> 5
0.00038128 fibonacci(6) -> 8
8


## Stacked decorators

In [ ]:
@d1
@d2
def f():
    print('f')

is the same as :

In [ ]:
def f():
    print('f')
    
f = d1(d2(f)

## Parameterized Decorators

##### Activate or deactivate the register decorator

In [46]:
registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
    
print('registry ->', registry)
f1()

running register(<function f1 at 0x7fc0fc39bd90>)
registry -> [<function f1 at 0x7fc0fc39bd90>]
running f1()


Let's try adding a parmeter

In [47]:
registry = set() #Adding and removing from a set is faster

def register(func, active = True):
    print(f'running register(active={active}) -> decorate({func})')
    if active:
        registry.add(func)
    else:
        registry.discard(func)
    return func

@register(active=False)
def f1():
    print('running f1()')
    
@register(active=True)
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
    
print('registry ->', registry)
f1()
f2()
f3()

TypeError: register() missing 1 required positional argument: 'func'

In [48]:
registry = set()

def register(active = True):
    def decorate(func):
        print(f'running register(active={active}) -> decorate({func})')
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

@register(active=False)
def f1():
    print('running f1()')
    
@register(active=True)
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
    
print('registry ->', registry)
f1()
f2()
f3()

running register(active=False) -> decorate(<function f1 at 0x7fc0fc2b4ea0>)
running register(active=True) -> decorate(<function f2 at 0x7fc0fc39bb70>)
registry -> {<function f2 at 0x7fc0fc39bb70>}
running f1()
running f2()
running f3()


Conceptually, the new register function is not a decorator but a decorator factory. When called, it returns the actual decorator that will be applied to the target function.

In [49]:
register()(f3)
print('registry ->', registry)

running register(active=True) -> decorate(<function f3 at 0x7fc0fc2b48c8>)
registry -> {<function f3 at 0x7fc0fc2b48c8>, <function f2 at 0x7fc0fc39bb70>}


--------------------------

But this parameterized decorator is simple, since it doesn't change the function. Parameterized decorators usually replace the decorated function, and their construction requires yet another level of nesting.

--------------------------

##### The clock decorator

In [50]:
import time

DEFAULT_FMT = '[{elapsed:.8f}] {name}({arg_str}) -> {result}'

def clock(fmt=DEFAULT_FMT):               # parameterized decorator factory
    def decorate(func):                   # actual decorator
        def clocked(*args):               # wraps the decorated function
            t0 = time.time()
            result = func(*args)
            elapsed = time.time() - t0
            name = func.__name__
            arg_str = ', '.join(repr(arg) for arg in args)
            print(fmt.format(**locals()))
            return result
        return clocked
    return decorate

In [51]:
@clock()
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

[0.12321639] snooze(0.123) -> None
[0.12320137] snooze(0.123) -> None
[0.12321329] snooze(0.123) -> None


In [52]:
@clock()
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

factorial(6)

[0.00000072] factorial(1) -> 1
[0.00007033] factorial(2) -> 2
[0.00009894] factorial(3) -> 6
[0.00012350] factorial(4) -> 24
[0.00018001] factorial(5) -> 120
[0.00042319] factorial(6) -> 720


720

In [53]:
@clock('{name}({arg_str}) dt={elapsed:.3f}')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.123
snooze(0.123) dt=0.123
snooze(0.123) dt=0.123
